In [10]:
symbols = ['BTC', 'ETH', 'BNB', 'XRP', 'SOL', 'DOT', 'LINK', 'DOGE']

# cryptocompare
high low open volumefrom volumeto close conversionType conversionSymbol date

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

def get_crypto_data(coin_symbol, start_date, end_date, interval='day', currency='USD'):
    """
    Fetch historical cryptocurrency data from CryptoCompare.

    Parameters:
    - coin_symbol: str (e.g., 'BTC', 'ETH')
    - start_date: str in 'YYYY-MM-DD'
    - end_date: str in 'YYYY-MM-DD'
    - interval: str - one of 'day', 'hour', 'minute'
    - currency: str - fiat currency (e.g., 'USD', 'EUR')

    Returns:
    - Pandas DataFrame with columns: time, open, high, low, close, volume_from, volume_to
    """

    interval_map = {
        'day': 'histoday',
        'hour': 'histohour',
        'minute': 'histominute'
    }

    if interval not in interval_map:
        raise ValueError("Interval must be one of 'day', 'hour', or 'minute'.")

    endpoint = f'https://min-api.cryptocompare.com/data/v2/{interval_map[interval]}'
    limit_per_call = 2000  # Max limit per API call
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    end_timestamp = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp())

    all_data = []
    toTs = end_timestamp

    while True:
        params = {
            'fsym': coin_symbol.upper(),
            'tsym': currency.upper(),
            'limit': limit_per_call - 1,  # API returns 'limit + 1' points
            'toTs': toTs
        }

        response = requests.get(endpoint, params=params)
        data = response.json()

        if data['Response'] != 'Success' or 'Data' not in data['Data']:
            raise Exception(f"API error: {data.get('Message', 'Unknown error')}")

        batch = data['Data']['Data']
        if not batch:
            break

        all_data = batch + all_data  # Prepend since we're going backwards
        oldest_time = batch[0]['time']

        if oldest_time <= start_timestamp:
            break

        toTs = oldest_time - 1
        time.sleep(0.25)  # Be polite to the API

    # Filter data to within range
    df = pd.DataFrame(all_data)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df[df['time'] >= pd.to_datetime(start_date)]
    df = df[df['time'] <= pd.to_datetime(end_date)]

    return df.reset_index(drop=True)

In [11]:
df = get_crypto_data('BTC', '2020-01-01', '2020-02-01', interval='day')
print(df.head())

        time     high      low     open  volumefrom      volumeto    close  \
0 2020-01-01  7267.16  7171.82  7183.88    11872.00  8.598619e+07  7189.94   
1 2020-01-02  7224.84  6945.83  7189.94    26531.32  1.873940e+08  6965.72   
2 2020-01-03  7407.11  6882.62  6965.72    51975.01  3.766904e+08  7339.70   
3 2020-01-04  7405.75  7278.60  7339.70    22083.57  1.621069e+08  7357.50   
4 2020-01-05  7501.42  7330.07  7357.50    24084.13  1.791477e+08  7359.96   

  conversionType conversionSymbol  
0         direct                   
1         direct                   
2         direct                   
3         direct                   
4         direct                   


In [12]:
df

,time,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol
0,2020-01-01,7267.16,7171.82,7183.88,11872.00,8.598619e+07,7189.94,direct,
1,2020-01-02,7224.84,6945.83,7189.94,26531.32,1.873940e+08,6965.72,direct,
2,2020-01-03,7407.11,6882.62,6965.72,51975.01,3.766904e+08,7339.70,direct,
3,2020-01-04,7405.75,7278.60,7339.70,22083.57,1.621069e+08,7357.50,direct,
4,2020-01-05,7501.42,7330.07,7357.50,24084.13,1.791477e+08,7359.96,direct,
5,2020-01-06,7799.63,7355.11,7359.96,46443.66,3.522862e+08,7762.69,direct,
6,2020-01-07,8218.63,7736.57,7762.69,75423.75,6.000545e+08,8160.36,direct,
7,2020-01-08,8458.65,7873.97,8160.36,85087.59,6.990043e+08,8045.15,direct,
8,2020-01-09,8047.11,7757.04,8045.15,40045.64,3.161995e+08,7817.35,direct,
9,2020-01-10,8192.05,7682.56,7817.35,55534.09,4.411316e+08,8190.31,direct,


# binance
open_time open high low close volume close_time quote_asset_volume number_of_trades taker_buy_base_vol taker_buy_quote_vol ignore 
date

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

def get_binance_data(symbol, start_date, end_date, interval='1d'):
    """
    Fetch historical kline/candlestick data from Binance API.

    Parameters:
    - symbol: str (e.g., 'BTCUSDT', 'ETHUSDT')
    - start_date: str in 'YYYY-MM-DD'
    - end_date: str in 'YYYY-MM-DD'
    - interval: str - Binance-supported interval (e.g., '1m', '5m', '1h', '1d')

    Returns:
    - Pandas DataFrame with columns: time, open, high, low, close, volume, etc.
    """

    base_url = 'https://api.binance.com/api/v3/klines'
    limit = 1000  # Max records per call
    interval_ms_map = {'1m': 60_000, '3m': 180_000, '5m': 300_000, '15m': 900_000, '30m': 1_800_000, '1h': 3_600_000, '2h': 7_200_000, '4h': 14_400_000, '6h': 21_600_000, '8h': 28_800_000, '12h': 43_200_000, '1d': 86_400_000, '3d': 259_200_000, '1w': 604_800_000, '1M': 2_592_000_000}

    if interval not in interval_ms_map:
        raise ValueError(f"Unsupported interval '{interval}'. Please use a Binance-supported interval.")

    start_ts = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp() * 1000)
    end_ts = int(datetime.strptime(end_date, '%Y-%m-%d').timestamp() * 1000)

    all_data = []

    while start_ts < end_ts:
        params = {'symbol': symbol.upper(), 'interval': interval, 'startTime': start_ts, 'endTime': end_ts, 'limit': limit}

        response = requests.get(base_url, params=params)
        response.raise_for_status()

        data = response.json()
        if not data:
            break

        all_data.extend(data)

        # Move to next batch
        last_time = data[-1][0]
        start_ts = last_time + interval_ms_map[interval]
        time.sleep(0.25)  # be kind to the API

    # Convert to DataFrame
    df = pd.DataFrame(all_data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

    # Process columns
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df['close_time'] = pd.to_datetime(df['close_time'], unit='ms')
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
    df[numeric_cols] = df[numeric_cols].astype(float)

    return df.drop(columns=['ignore']).reset_index(drop=True)

In [ ]:
df_daily = get_binance_data('BTCUSDT', '2017-01-01', '2026-01-01', interval='1d')
# df_4hour = get_binance_data('BTCUSDT', '2017-01-01', '2026-01-01', interval='4h')

# Yahoo Finance (4h interval için sadece geçmiş 2 yılın datası var)
Close High Low Open Volume date

In [ ]:
import yfinance as yf

def fetch_from_yfinance(symbol='ETH-USD', period="max"):
    df = yf.download(symbol, period=period, interval='1d')
    df.reset_index(inplace=True)
    df["date"] = df["Date"].dt.date
    df.drop(columns=["Date"], inplace=True)
    
    print(f"\n[Yahoo Finance - {symbol}]")
    print(f"Records: {len(df)}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(df.tail())
    return df[["date", "Close"]]

symbols = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'SOL-USD', 'DOT-USD', 'LINK-USD', 'DOGE-USD']
yfinance_data = {s: fetch_from_yfinance(s) for s in symbols}


In [ ]:
import yfinance as yf

def fetch_from_yfinance(symbol='ETH-USD', period="max"):
    df = yf.download(symbol, period="1y", interval='4h')
    df.reset_index(inplace=True)
    df["date"] = df["Date"].dt.date
    df.drop(columns=["Date"], inplace=True)

    print(f"\n[Yahoo Finance - {symbol}]")
    print(f"Records: {len(df)}")
    print(f"Date range: {df['date'].min()} to {df['date'].max()}")
    print(df.tail())
    return df

symbols = ['BTC-USD']
yfinance_data = {s: fetch_from_yfinance(s) for s in symbols}

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (4h 1926-05-28 15:48:53+00:00 -> 2025-05-03 15:48:53+00:00) (Yahoo error = "4h data not available for startTime=1410912000 and endTime=1746287333. The requested range must be within the last 730 days.")')



[Yahoo Finance - BTC-USD]
Records: 0
Date range: nan to nan
Empty DataFrame
Columns: [(Adj Close, BTC-USD), (Close, BTC-USD), (High, BTC-USD), (Low, BTC-USD), (Open, BTC-USD), (Volume, BTC-USD), (date, )]
Index: []


/tmp/ipykernel_23385/3997505157.py:7: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["Date"], inplace=True)


In [ ]:
yfinance_data["BTC-USD"]

Price,Close,High,Low,Open,Volume,date
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD,
0,457.334015,468.174011,452.421997,465.864014,21056800,2014-09-17
1,424.440002,456.859985,413.104004,456.859985,34483200,2014-09-18
2,394.795990,427.834991,384.532013,424.102997,37919700,2014-09-19
3,408.903992,423.295990,389.882996,394.673004,36863600,2014-09-20
4,398.821014,412.425995,393.181000,408.084991,26580100,2014-09-21
...,...,...,...,...,...,...
3877,94284.789062,95485.414062,93796.632812,94981.859375,25806129921,2025-04-29
3878,94207.312500,95249.320312,92979.640625,94286.468750,28344679831,2025-04-30
3879,96492.335938,97437.960938,94153.632812,94212.859375,32875889623,2025-05-01


In [ ]:
"""
[{'time': 1573430400,
  'high': 9084.49,
  'low': 8619.78,
  'open': 9045.85,
  'volumefrom': 25750.4,
  'volumeto': 226035620.33,
  'close': 8726.31,
  'conversionType': 'direct',
  'conversionSymbol': ''},
"""
len(yfinance_data["BTC-USD"])

In [38]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd  # Import pandas

def fetch_4hourly_by_range(symbol='ETH-USD', start_date='2024-01-01', end_date='2024-03-31'):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    all_4hourly_data = []
    interval = '4h'

    current_start = start
    while current_start < end:
        current_end_dt = current_start + timedelta(days=60)  # Still using 60-day chunks for intraday
        if current_end_dt > end:
            current_end_dt = end
        current_end_str = current_end_dt.strftime('%Y-%m-%d')
        current_start_str = current_start.strftime('%Y-%m-%d')

        print(f"\nFetching 4-hourly data for {symbol} from {current_start_str} to {current_end_str}")
        df = yf.download(symbol, start=current_start_str, end=current_end_str, interval=interval)

        if not df.empty:
            df.reset_index(inplace=True)
            df["date"] = df["Datetime"].dt.date
            df["hour"] = df["Datetime"].dt.hour
            df.drop(columns=["Datetime"], inplace=True)
            all_4hourly_data.append(df[["date", "hour", "Open", "High", "Low", "Close"]])
            print(f"  Records in this chunk: {len(df)}")
            print(f"  Date range: {df['date'].min()} to {df['date'].max()}")
            print(df.tail())
        else:
            print("  No data found for this period.")

        current_start = current_end_dt

    if all_4hourly_data:
        combined_df = pd.concat(all_4hourly_data)
        print(f"\n[Yahoo Finance - {symbol} - 4-Hourly - Combined]")
        print(f"Total Records: {len(combined_df)}")
        print(f"Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")
        print(combined_df.head())
        print(combined_df.tail())
        return combined_df
    else:
        print(f"\n[Yahoo Finance - {symbol} - 4-Hourly]")
        print("No 4-hourly data fetched for the specified range.")
        return None

symbols = ['BTC-USD', 'ETH-USD']  # Limiting to two symbols for a shorter example
yfinance_4hourly_data_range = {s: fetch_4hourly_by_range(s, start_date='2024-01-01', end_date='2024-03-31') for s in symbols}

# The yfinance_4hourly_data_range dictionary will now contain DataFrames (or None)
# with 4-hourly data for BTC-USD and ETH-USD between January 1, 2024, and March 31, 2024.
# You can access the data like this:
# if 'BTC-USD' in yfinance_4hourly_data_range and yfinance_4hourly_data_range['BTC-USD'] is not None:
#     print("\nFirst few rows of BTC-USD 4-hourly data:")
#     print(yfinance_4hourly_data_range['BTC-USD'].head())


Fetching 4-hourly data for BTC-USD from 2024-01-01 to 2024-03-01


[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_23385/2738586738.py:26: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["Datetime"], inplace=True)
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_23385/2738586738.py:26: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["Datetime"], inplace=True)


  Records in this chunk: 360
  Date range: 2024-01-01 to 2024-02-29
Price          Close          High           Low          Open      Volume  \
Ticker       BTC-USD       BTC-USD       BTC-USD       BTC-USD     BTC-USD   
355     62680.195312  63585.644531  61359.929688  61558.410156  6986833920   
356     62945.632812  63296.148438  62222.050781  62688.980469           0   
357     62829.769531  63544.921875  62215.234375  62924.363281           0   
358     61568.136719  62888.285156  60498.730469  62796.378906           0   
359     61297.058594  62280.144531  60684.500000  61590.046875   723378176   

Price         date hour  
Ticker                   
355     2024-02-29    4  
356     2024-02-29    8  
357     2024-02-29   12  
358     2024-02-29   16  
359     2024-02-29   20  

Fetching 4-hourly data for BTC-USD from 2024-03-01 to 2024-03-31
  Records in this chunk: 180
  Date range: 2024-03-01 to 2024-03-30
Price          Close          High           Low          Open     Vo

[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_23385/2738586738.py:26: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["Datetime"], inplace=True)
[*********************100%***********************]  1 of 1 completed

  Records in this chunk: 360
  Date range: 2024-01-01 to 2024-02-29
Price         Close         High          Low         Open      Volume  \
Ticker      ETH-USD      ETH-USD      ETH-USD      ETH-USD     ETH-USD   
355     3467.877197  3492.261475  3400.451904  3407.055420  1763594240   
356     3515.835449  3518.969238  3446.039062  3467.748535   158826496   
357     3469.679199  3514.480713  3453.982666  3513.301270   100732928   
358     3403.936523  3469.500000  3354.909424  3469.500000           0   
359     3346.343506  3414.987061  3303.905273  3403.201904   714622976   

Price         date hour  
Ticker                   
355     2024-02-29    4  
356     2024-02-29    8  
357     2024-02-29   12  
358     2024-02-29   16  
359     2024-02-29   20  

Fetching 4-hourly data for ETH-USD from 2024-03-01 to 2024-03-31
  Records in this chunk: 180
  Date range: 2024-03-01 to 2024-03-30
Price         Close         High          Low         Open     Volume  \
Ticker      ETH-USD     


/tmp/ipykernel_23385/2738586738.py:26: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=["Datetime"], inplace=True)


In [39]:
yfinance_4hourly_data_range["BTC-USD"]

Price,date,hour,Open,High,Low,Close
Ticker,,,BTC-USD,BTC-USD,BTC-USD,BTC-USD
0,2024-01-01,0,42280.234375,42718.710938,42270.757812,42320.730469
1,2024-01-01,4,42325.042969,42506.726562,42214.976562,42466.468750
2,2024-01-01,8,42475.914062,42759.218750,42464.808594,42687.308594
3,2024-01-01,12,42687.914062,42831.003906,42594.304688,42780.917969
4,2024-01-01,16,42787.179688,43544.480469,42686.406250,43529.839844
...,...,...,...,...,...,...
175,2024-03-30,4,69815.187500,70342.195312,69807.390625,69902.726562
176,2024-03-30,8,69893.171875,70206.546875,69856.835938,70206.546875
177,2024-03-30,12,70216.289062,70279.226562,69992.578125,70049.546875


In [40]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

def fetch_hourly_by_year(symbol='ETH-USD', start_year=2020, end_year=2025):
    all_hourly_data = []
    for year in range(start_year, end_year + 1):
        start_date = datetime(year, 1, 1)
        end_date = datetime(year + 1, 1, 1) - timedelta(days=1)  # Last day of the year

        current_start = start_date
        while current_start <= end_date:
            current_end_dt = current_start + timedelta(days=60)
            if current_end_dt > end_date:
                current_end_dt = end_date

            current_start_str = current_start.strftime('%Y-%m-%d')
            current_end_str = current_end_dt.strftime('%Y-%m-%d')

            print(f"\nFetching hourly data for {symbol} from {current_start_str} to {current_end_str}")
            df = yf.download(symbol, start=current_start_str, end=current_end_str, interval='1h')

            if not df.empty:
                df.reset_index(inplace=True)
                df["date"] = df["Datetime"].dt.date
                df["hour"] = df["Datetime"].dt.hour
                df.drop(columns=["Datetime"], inplace=True)
                all_hourly_data.append(df[["date", "hour", "Open", "High", "Low", "Close"]])
                print(f"  Records in this chunk: {len(df)}")
                print(f"  Date range: {df['date'].min()} to {df['date'].max()}")
                print(df.tail())
            else:
                print("  No data found for this period.")

            current_start = current_end_dt + timedelta(days=1) # Move to the next day

    if all_hourly_data:
        combined_df = pd.concat(all_hourly_data)
        print(f"\n[Yahoo Finance - {symbol} - Hourly - {start_year} to {end_year}]")
        print(f"Total Records: {len(combined_df)}")
        print(f"Date range: {combined_df['date'].min()} to {combined_df['date'].max()}")
        print(combined_df.head())
        print(combined_df.tail())
        return combined_df
    else:
        print(f"\n[Yahoo Finance - {symbol} - Hourly - {start_year} to {end_year}]")
        print("No hourly data fetched for the specified range.")
        return None

symbols = ['BTC-USD', 'ETH-USD'] # Example with two symbols
all_symbols_hourly_data = {}
start_year = 2020
end_year = 2025

for symbol in symbols:
    print(f"\n--- Fetching hourly data for {symbol} from {start_year} to {end_year} ---")
    hourly_data = fetch_hourly_by_year(symbol, start_year, end_year)
    all_symbols_hourly_data[symbol] = hourly_data

# The all_symbols_hourly_data dictionary will now contain DataFrames (or None)
# for each symbol, with hourly data from 2020 to 2025 (as much as available
# within the 60-day intraday limit).
# You can access the data for a specific symbol like this:
# if 'BTC-USD' in all_symbols_hourly_data and all_symbols_hourly_data['BTC-USD'] is not None:
#     print("\nFirst few rows of BTC-USD hourly data (2020-2025):")
#     print(all_symbols_hourly_data['BTC-USD'].head())


--- Fetching hourly data for BTC-USD from 2020 to 2025 ---

Fetching hourly data for BTC-USD from 2020-01-01 to 2020-03-01


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (1h 2020-01-01 -> 2020-03-01) (Yahoo error = "1h data not available for startTime=1577836800 and endTime=1583020800. The requested range must be within the last 730 days.")')


  No data found for this period.

Fetching hourly data for BTC-USD from 2020-03-02 to 2020-05-01


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (1h 2020-03-02 -> 2020-05-01) (Yahoo error = "1h data not available for startTime=1583107200 and endTime=1588291200. The requested range must be within the last 730 days.")')


  No data found for this period.

Fetching hourly data for BTC-USD from 2020-05-02 to 2020-07-01


...

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (1h 2022-11-02 -> 2022-12-31) (Yahoo error = "1h data not available for startTime=1667347200 and endTime=1672444800. The requested range must be within the last 730 days.")')


  No data found for this period.

Fetching hourly data for BTC-USD from 2023-01-01 to 2023-03-02


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: YFPricesMissingError('possibly delisted; no price data found  (1h 2023-01-01 -> 2023-03-02) (Yahoo error = "1h data not available for startTime=1672531200 and endTime=1677715200. The requested range must be within the last 730 days.")')


  No data found for this period.

Fetching hourly data for BTC-USD from 2023-03-03 to 2023-05-02


KeyboardInterrupt: 